In [ ]:
!rm -rf real-video-enhancer/
!git clone https://github.com/tntwise/real-video-enhancer

In [ ]:
# https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/ is where all model files for RVE are stored
# you can download them from there, or try custom models from https://openmodeldb.info/
# Use .pth models for upscaling, and rife .pkl for interpolation, GMFSS and GIMM will also work from the repo
!mkdir real-video-enhancer/models/
!cd real-video-enhancer/models/ && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/rife4.26.pkl
!cd real-video-enhancer/models/ && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/rife4.22.pkl
!cd real-video-enhancer/models/ && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/rife4.22-lite.pkl # this is the fastest model
!cd real-video-enhancer/models/ && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/2x_ModernSpanimationV2.pth



In [ ]:
!pip install  --pre --extra-index-url https://download.pytorch.org/whl/nightly/cu126 -r real-video-enhancer/backend/requirements.txt
!cd real-video-enhancer && mkdir bin/ && cd bin && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/ffmpeg-colab/ffmpeg && chmod +x ffmpeg



```
usage: rve-backend.py [-h] [-i INPUT] [-o OUTPUT] [-l OVERLAP] [-b BACKEND]
                      [--upscale_model UPSCALE_MODEL]
                      [--interpolate_model INTERPOLATE_MODEL]
                      [--interpolate_factor INTERPOLATE_FACTOR]
                      [--precision PRECISION]
                      [--tensorrt_opt_profile TENSORRT_OPT_PROFILE]
                      [--scene_detect_method SCENE_DETECT_METHOD]
                      [--scene_detect_threshold SCENE_DETECT_THRESHOLD]
                      [--overwrite] [--border_detect] [--crf CRF]
                      [--video_encoder_preset {libx264,libx265,vp9,av1,x264_vulkan,x264_nvenc,x265_nvenc,av1_nvenc}]
                      [--audio_encoder_preset {aac,libmp3lame}]
                      [--audio_bitrate AUDIO_BITRATE]
                      [--custom_encoder CUSTOM_ENCODER] [--tilesize TILESIZE]
                      [--pytorch_gpu_id PYTORCH_GPU_ID]
                      [--ncnn_gpu_id NCNN_GPU_ID] [--benchmark] [--UHD_mode]
                      [--slomo_mode] [--dynamic_scaled_optical_flow]
                      [--ensemble] [--shared_memory_id SHARED_MEMORY_ID]
                      [--list_backends] [--paused_file PAUSED_FILE]
                      [--upscale_output_resolution UPSCALE_OUTPUT_RESOLUTION]

Backend to RVE, used to upscale and interpolate videos

options:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        input video path
  -o OUTPUT, --output OUTPUT
                        output video path or PIPE
  -l OVERLAP, --overlap OVERLAP
                        overlap size on tiled rendering (default=10)
  -b BACKEND, --backend BACKEND
                        backend used to upscale image.
                        (pytorch/ncnn/tensorrt/directml, default=pytorch)
  --upscale_model UPSCALE_MODEL
                        Direct path to upscaling model, will automatically
                        upscale if model is valid.
  --interpolate_model INTERPOLATE_MODEL
                        Direct path to interpolation model, will automatically
                        interpolate if model is valid. (Downloadable Options:
                        [rife46, rife47, rife415, rife418, rife420, rife422,
                        rife422lite]))
  --interpolate_factor INTERPOLATE_FACTOR
                        Multiplier for interpolation, will round up to nearest
                        integer for interpolation but the fps will be correct
  --precision PRECISION
                        sets precision for model, (auto/float16/float32,
                        default=auto)
  --tensorrt_opt_profile TENSORRT_OPT_PROFILE
                        sets tensorrt optimization profile for model,
                        (1/2/3/4/5, default=3)
  --scene_detect_method SCENE_DETECT_METHOD
                        Scene change detection to avoid interpolating
                        transitions. (options=mean, mean_segmented, none) Mean
                        segmented splits up an image, and if an arbitrary
                        number of segments changes are detected within the
                        segments, it will trigger a scene change. (lower
                        sensativity thresholds are not recommended)
  --scene_detect_threshold SCENE_DETECT_THRESHOLD
                        Scene change detection sensitivity, lower number means
                        it has a higher chance of detecting scene changes,
                        with risk of detecting too many.
  --overwrite           Overwrite output video if it already exists.
  --border_detect       Detects current borders and removes them, useful for
                        removing black bars.
  --crf CRF             Constant rate factor for videos, lower setting means
                        higher quality.
  --video_encoder_preset {libx264,libx265,vp9,av1,x264_vulkan,x264_nvenc,x265_nvenc,av1_nvenc}
                        encoder preset that sets default encoder settings
                        useful for hardware encoders.
  --audio_encoder_preset {aac,libmp3lame}
                        encoder preset that sets default encoder settings
  --audio_bitrate AUDIO_BITRATE
                        bitrate for audio if preset is used
  --custom_encoder CUSTOM_ENCODER
                        custom encoder
  --tilesize TILESIZE   upscale images in smaller chunks, default is the size
                        of the input video
  --pytorch_gpu_id PYTORCH_GPU_ID
                        GPU ID for pytorch backend, default is 0
  --ncnn_gpu_id NCNN_GPU_ID
                        GPU ID for ncnn backend, default is 0
  --benchmark           Benchmark without saving video
  --UHD_mode            Lowers the resoltion flow is calculated at, speeding
                        up model and saving vram. Helpful for higher
                        resultions.
  --slomo_mode          Instead of increasing framerate, it will remain the
                        same while just increasing the length of the video.
  --dynamic_scaled_optical_flow
                        Scale the optical flow based on the difference between
                        frames, currently only works with the pytorch backend.
  --ensemble            Use ensemble when interpolating if the model supports
                        it.
  --shared_memory_id SHARED_MEMORY_ID
                        Memory ID to share preview ons
  --list_backends       list out available backends
  --paused_file PAUSED_FILE
                        File to store paused state (True means paused, False
                        means unpaused)
  --upscale_output_resolution UPSCALE_OUTPUT_RESOLUTION
                        Resolution of output video, this is helpful for 4x
                        models when you only want 2x upscaling. Ex:
                        (1920x1080)
```





In [ ]:
#example upscale/denoise command
# It is recommended to use  --video_encoder_preset x264_nvenc
!cd real-video-enhancer/ && python3 backend/rve-backend.py -i ../{INPUT_FILE} -o ../{OUTPUT_FILE} --upscale_model models/{MODEL} --backend pytorch --video_encoder_preset x264_nvenc

In [ ]:
# example interpoalte command
# It is recommended to use  --video_encoder_preset x264_nvenc
!cd real-video-enhancer/ && python3 backend/rve-backend.py -i ../{INPUT_FILE} -o ../{OUTPUT_FILE} --interpolate_model models/{MODEL} --backend pytorch --interpolate_factor 2 --video_encoder_preset x264_nvenc